# Folium TimestampedGeoJson Demo

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

This notebook demonstrates how to combine MovingPandas and the **Folium [TimestampedGeoJson](https://python-visualization.github.io/folium/plugins.html#folium.plugins.TimestampedGeoJson)** plugin to create an animated trajectory visualization. 

*This approach was proposed by [David Bailey](https://github.com/anitagraser/movingpandas/issues/103)*.

In [ ]:
%matplotlib inline

In [ ]:
import urllib
import os
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from pyproj import CRS
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import movingpandas as mpd
import folium
from folium.plugins import TimestampedGeoJson
import random

import warnings
warnings.simplefilter("ignore")

import panel as pn
import panel.widgets as pnw

In [ ]:
df = read_file('../data/geolife_small.gpkg')
df['t'] = pd.to_datetime(df['t'])
df = df.set_index('t').tz_localize(None)

In [ ]:
trajs = mpd.TrajectoryCollection(df, 'trajectory_id')
trajs = mpd.MinTimeDeltaGeneralizer(trajs).generalize(tolerance=timedelta(minutes=1))

In [ ]:
my_traj = trajs.trajectories[1]
my_traj.plot()

In [ ]:
def traj_to_timestamped_geojson(trajectory):
    features = []
    df = trajectory.df.copy()
    df["previous_geometry"] = df["geometry"].shift()
    df["time"] = df.index
    df["previous_time"] = df["time"].shift()
    for _, row in df.iloc[1:].iterrows():
        coordinates = [
            [
                row["previous_geometry"].xy[0][0],
                row["previous_geometry"].xy[1][0]
            ],
            [
                row["geometry"].xy[0][0],
                row["geometry"].xy[1][0]
            ]
        ]
        times = [row["previous_time"].isoformat(), row["time"].isoformat()]
        features.append(
            {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": coordinates,
                },
                "properties": {
                    "times": times,
                    "style": {
                        "color": '#2196f3',
                        "weight": 5,
                    },
                },
            }
        )
    return features


features = traj_to_timestamped_geojson(my_traj)
xy = my_traj.get_start_location()
m = folium.Map(location=[xy.y, xy.x], zoom_start=14)

TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1S",
    add_last_point=True,
    transition_time=10
).add_to(m)

m